In [52]:
import numpy as np
import matplotlib.pyplot as plt

In [53]:
class Jackknife:
    def __init__( self, len_data, binsize ):
        self.binsize = binsize
        self.nbins = int(len_data/self.binsize)
        self.N = self.binsize * self.nbins
        self.jack_avg = []
        self.est = 0
        self.var_est = 0

    def set( self, func, list_of_data ):
        for i in range( self.nbins ):
            self.jack_avg.append( func( i, self.binsize, list_of_data ) )

    def do_it( self ):
        for i in range( 0, self.nbins ):
            self.est += self.jack_avg[i]
        self.est /= self.nbins

        for i in range( 0, self.nbins ):
            self.var_est += ( self.jack_avg[i] - self.est )**2
        self.var_est /= self.nbins
        self.var_est *= self.nbins -1

    def mean( self ):
        return self.est

    def var( self ):
        return self.var_est

    def err( self ):
        return np.sqrt(self.var_est)

def simple_mean(i, binsize, np_data):
    resmpld = np.delete(np_data, np.s_[i*binsize:(i+1)*binsize], axis=0)
    return np.mean(resmpld, axis=0)

def format_print(cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'

def format_print_w_exact(exact, cen, err):
    if np.abs(err)<1.0e-15:
        return str(cen)+" exact:"+str(exact)
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'+' exact:'+'{ex:.{width}f}'.format(ex=exact, width=i+2)+' ['+'{num:.{width}f}'.format(num=abs(exact-cen)/err, width=2)+' sigma]'

In [54]:
def format_print_w_exact_matrix(exact, cen, err):
    shape=exact.shape
    res=np.full(shape, "", dtype=object)

    for i in np.arange(shape[0]):
        for j in np.arange(shape[1]):
            res[i,j]=format_print_w_exact(exact[i,j], cen[i,j], err[i,j])
            
    return np.array(res)

In [55]:
def H3x3_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4,s5,s6,s7,s8,s9):
    res = 0.0
    res += kx_*s1*s2
    res += kx_*s2*s3
    res += kx_*s3*s1
    res += kx_*s4*s5
    res += kx_*s5*s6
    res += kx_*s6*s4
    res += kx_*s7*s8
    res += kx_*s8*s9
    res += kx_*s9*s7
    #
    res += ky_*s1*s4
    res += ky_*s2*s5
    res += ky_*s3*s6
    res += ky_*s4*s7
    res += ky_*s5*s8
    res += ky_*s6*s9
    res += ky_*s7*s1
    res += ky_*s8*s2
    res += ky_*s9*s3
    #
    res += kz_*s1*s5
    res += kz_*s2*s6
    res += kz_*s3*s4
    res += kz_*s4*s8
    res += kz_*s5*s9
    res += kz_*s6*s7
    res += kz_*s7*s2
    res += kz_*s8*s3
    res += kz_*s9*s1
    
    res *= -beta_
    res -= h_*(s1+s2+s3+s4+s5+s6+s7+s8+s9)
    
    return res

In [56]:
def Z3x3_numeric(beta_,h_, kx_,ky_,kz_):
    res=0.0
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        res += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                    s1,s2,s3,s4,s5,s6,s7,s8,s9))
    return res

In [57]:
def mag_per_site3x3_numeric(beta_,h_, kx_,ky_,kz_):
    numer=0.0
    denom=0.0
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        numer += (s1+s2+s3+s4+s5+s6+s7+s8+s9)*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                                     s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))
    res = numer/denom/9.0
    return res

In [58]:
def ss_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    mat=np.array([
        [s1*s1, s1*s2, s1*s3],
        [s1*s4, s1*s5, s1*s6],
        [s1*s7, s1*s8, s1*s9]
    ])
    return mat

In [59]:
def exex_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ex=np.array([
        [s1*s2, s2*s3, s3*s1],
        [s4*s5, s5*s6, s6*s4],
        [s7*s8, s8*s9, s9*s7]
    ])
    mat=np.array([
        [ex[0,0]*ex[0,0], ex[0,0]*ex[0,1], ex[0,0]*ex[0,2]],
        [ex[0,0]*ex[1,0], ex[0,0]*ex[1,1], ex[0,0]*ex[1,2]],
        [ex[0,0]*ex[2,0], ex[0,0]*ex[2,1], ex[0,0]*ex[2,2]]
    ])
    return mat

In [60]:
def exey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ex=np.array([
        [s1*s2, s2*s3, s3*s1],
        [s4*s5, s5*s6, s6*s4],
        [s7*s8, s8*s9, s9*s7]
    ])
    ey=np.array([
        [s1*s4, s2*s5, s3*s6],
        [s4*s7, s5*s8, s6*s9],
        [s7*s1, s8*s2, s9*s3]
    ])
    mat=np.array([
        [ex[0,0]*ey[0,0], ex[0,0]*ey[0,1], ex[0,0]*ey[0,2]],
        [ex[0,0]*ey[1,0], ex[0,0]*ey[1,1], ex[0,0]*ey[1,2]],
        [ex[0,0]*ey[2,0], ex[0,0]*ey[2,1], ex[0,0]*ey[2,2]]
    ])
    return mat

In [61]:
def exez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ex=np.array([
        [s1*s2, s2*s3, s3*s1],
        [s4*s5, s5*s6, s6*s4],
        [s7*s8, s8*s9, s9*s7]
    ])
    ez=np.array([
        [s1*s5, s2*s6, s3*s4],
        [s4*s8, s5*s9, s6*s7],
        [s7*s2, s8*s3, s9*s1]
    ])
    mat=np.array([
        [ex[0,0]*ez[0,0], ex[0,0]*ez[0,1], ex[0,0]*ez[0,2]],
        [ex[0,0]*ez[1,0], ex[0,0]*ez[1,1], ex[0,0]*ez[1,2]],
        [ex[0,0]*ez[2,0], ex[0,0]*ez[2,1], ex[0,0]*ez[2,2]]
    ])
    return mat

In [62]:
def eyey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ey=np.array([
        [s1*s4, s2*s5, s3*s6],
        [s4*s7, s5*s8, s6*s9],
        [s7*s1, s8*s2, s9*s3]
    ])
    mat=np.array([
        [ey[0,0]*ey[0,0], ey[0,0]*ey[0,1], ey[0,0]*ey[0,2]],
        [ey[0,0]*ey[1,0], ey[0,0]*ey[1,1], ey[0,0]*ey[1,2]],
        [ey[0,0]*ey[2,0], ey[0,0]*ey[2,1], ey[0,0]*ey[2,2]]
    ])
    return mat

In [63]:
def eyez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ey=np.array([
        [s1*s4, s2*s5, s3*s6],
        [s4*s7, s5*s8, s6*s9],
        [s7*s1, s8*s2, s9*s3]
    ])
    ez=np.array([
        [s1*s5, s2*s6, s3*s4],
        [s4*s8, s5*s9, s6*s7],
        [s7*s2, s8*s3, s9*s1]
    ])
    mat=np.array([
        [ey[0,0]*ez[0,0], ey[0,0]*ez[0,1], ey[0,0]*ez[0,2]],
        [ey[0,0]*ez[1,0], ey[0,0]*ez[1,1], ey[0,0]*ez[1,2]],
        [ey[0,0]*ez[2,0], ey[0,0]*ez[2,1], ey[0,0]*ez[2,2]]
    ])
    return mat

In [64]:
def ezez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ez=np.array([
        [s1*s5, s2*s6, s3*s4],
        [s4*s8, s5*s9, s6*s7],
        [s7*s2, s8*s3, s9*s1]
    ])
    mat=np.array([
        [ez[0,0]*ez[0,0], ez[0,0]*ez[0,1], ez[0,0]*ez[0,2]],
        [ez[0,0]*ez[1,0], ez[0,0]*ez[1,1], ez[0,0]*ez[1,2]],
        [ez[0,0]*ez[2,0], ez[0,0]*ez[2,1], ez[0,0]*ez[2,2]]
    ])
    return mat

In [65]:
def ss_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=ss_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = ss_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [66]:
def exex_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exex_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = exex_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [67]:
def exey_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exey_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = exey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [68]:
def exez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exez_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = exez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [69]:
def eyey_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=eyey_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = eyey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [70]:
def eyez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=eyez_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = eyez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [71]:
def ezez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=ezez_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = ezez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [72]:
beta=0.0746531
h=0.

kx=1.2
ky=1.5
kz=0.3

# kx=1.0
# ky=1.0
# kz=1.0

In [124]:
nin=0
nfin=10
nint=1
binsize=1

In [125]:
mag_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/mag/"+str(n)+".dat")
    mag_.append(dat)
    
mag=np.array(mag_)

In [126]:
jk_mag = Jackknife(mag.size, binsize)
jk_mag.set(simple_mean, mag)
jk_mag.do_it()

In [127]:
print("mag_per_site")
format_print_w_exact(0.0, jk_mag.mean(), jk_mag.err())

mag_per_site


'1.2(23) exact:0.00 [0.51 sigma]'

In [128]:
jk_mag.mean()

1.2

In [129]:
# delta=1.0e-3
# h=1.0

# print("mag")
# numeric=(np.log(Z3x3_numeric(beta, h+delta, kx,ky,kz))-np.log(Z3x3_numeric(beta, h-delta, kx,ky,kz)))/(2.0*delta)/9.0
# exact=mag_per_site3x3_numeric(beta,h, kx,ky,kz)
# print(format_print_w_exact(exact, numeric, delta**2))

In [130]:
h=0.0

In [133]:
mag

array([  6.,   8.,   6.,  10.,   2.,  -2.,   0.,  -2.,   0., -16.])

In [134]:
mag*mag

array([ 36.,  64.,  36., 100.,   4.,   4.,   0.,   4.,   0., 256.])

In [131]:
np.mean(mag**2/32)

1.575

In [132]:
jk_magsq = Jackknife(mag.size, binsize)
jk_magsq.set(simple_mean, mag**2/32)
jk_magsq.do_it()

In [118]:
# print("susc")
# numeric=(mag_per_site3x3_numeric(beta,h+delta, kx,ky,kz)-mag_per_site3x3_numeric(beta,h-delta, kx,ky,kz))/(2.0*delta)
# format_print(numeric, jk_magsq.mean(), jk_magsq.err())

In [119]:
print("susc")
format_print(jk_magsq.mean(), jk_magsq.err())

susc


'1.57(79)'

In [104]:
jk_magsq.mean()

1.5749999999999997

In [105]:
# delta=1.0e-3
# print("mag")
# numeric=-(np.log(Z2x2(beta, h+delta, kx,ky,kz))-np.log(Z2x2(beta, h-delta, kx,ky,kz)))/(2.0*delta)/4.0
# exact=mag_per_site2x2(beta,h, kx,ky,kz)
# print(format_print_w_exact(exact, numeric, delta**2))
# print("susc")
# numeric=-(mag_per_site2x2(beta,h+delta, kx,ky,kz)-mag_per_site2x2(beta,h-delta, kx,ky,kz))/(2.0*delta)
# exact=susc2x2(beta,h, kx,ky,kz)
# print(format_print_w_exact(exact, numeric, delta**2))

In [106]:
ss_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/ss/"+str(n)+".dat")
    ss_.append(dat)
    
ss=np.array(ss_)

jk_ss = Jackknife(ss.size, binsize)
jk_ss.set(simple_mean, ss)
jk_ss.do_it()

print(format_print_w_exact_matrix(ss_corr_numeric(beta, h, kx,ky,kz), jk_ss.mean(), jk_ss.err()))

[['1.0 exact:1.0' '0.013(31) exact:0.1089 [3.14 sigma]'
  '-0.175(71) exact:0.1089 [3.98 sigma]']
 ['0.163(48) exact:0.1333 [0.61 sigma]'
  '-0.062(49) exact:0.0532 [2.36 sigma]'
  '-0.013(57) exact:0.0387 [0.90 sigma]']
 ['-0.087(56) exact:0.1333 [3.97 sigma]'
  '0.062(56) exact:0.0387 [0.42 sigma]'
  '-0.050(45) exact:0.0532 [2.31 sigma]']]


In [107]:
jk_ss.mean()

array([[ 1.    ,  0.0125, -0.175 ,  0.0125],
       [ 0.1625, -0.0625, -0.0125,  0.15  ],
       [-0.0875,  0.0625, -0.05  ,  0.075 ],
       [ 0.05  , -0.1625,  0.    ,  0.1875],
       [-0.025 , -0.0375,  0.2   , -0.0375],
       [ 0.05  ,  0.1875,  0.    , -0.1625],
       [-0.0875,  0.075 , -0.05  ,  0.0625],
       [ 0.1625,  0.15  , -0.0125, -0.0625]])

In [108]:
exex_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exex/"+str(n)+".dat")
    exex_.append(dat)
    
exex=np.array(exex_)

jk_exex = Jackknife(exex.size, binsize)
jk_exex.set(simple_mean, exex)
jk_exex.do_it()

print(format_print_w_exact_matrix(exex_corr_numeric(beta, h, kx,ky,kz), jk_exex.mean(), jk_exex.err()))

[['1.0 exact:1.0' '-0.175(71) exact:0.1089 [3.98 sigma]'
  '0.000(68) exact:0.1089 [1.60 sigma]']
 ['-0.113(69) exact:0.0281 [2.03 sigma]'
  '0.225(33) exact:0.0167 [6.35 sigma]'
  '-0.113(41) exact:0.0181 [3.16 sigma]']
 ['0.138(50) exact:0.0281 [2.20 sigma]'
  '-0.188(35) exact:0.0181 [5.81 sigma]'
  '0.062(60) exact:0.0167 [0.77 sigma]']]


In [109]:
jk_exex.mean()

array([[ 1.    , -0.175 ,  0.    , -0.175 ],
       [-0.1125,  0.225 , -0.1125,  0.    ],
       [ 0.1375, -0.1875,  0.0625, -0.0625],
       [-0.025 ,  0.025 , -0.1   ,  0.1   ],
       [ 0.15  , -0.175 , -0.1   , -0.175 ],
       [-0.025 ,  0.1   , -0.1   ,  0.025 ],
       [ 0.1375, -0.0625,  0.0625, -0.1875],
       [-0.1125,  0.    , -0.1125,  0.225 ]])

In [96]:
exey_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exey/"+str(n)+".dat")
    exey_.append(dat)
    
exey=np.array(exey_)

jk_exey = Jackknife(exey.size, binsize)
jk_exey.set(simple_mean, exey)
jk_exey.do_it()

print(format_print_w_exact_matrix(exey_corr_numeric(beta, h, kx,ky,kz), jk_exey.mean(), jk_exey.err()))

[['0.150(18) exact:0.0387 [6.35 sigma]'
  '-0.062(49) exact:0.0532 [2.36 sigma]'
  '0.113(24) exact:0.0199 [3.92 sigma]']
 ['0.075(40) exact:0.0209 [1.34 sigma]'
  '0.100(67) exact:0.0209 [1.17 sigma]'
  '-0.188(49) exact:0.0170 [4.17 sigma]']
 ['0.100(26) exact:0.0532 [1.78 sigma]'
  '-0.062(29) exact:0.0387 [3.44 sigma]'
  '0.100(55) exact:0.0199 [1.46 sigma]']]


In [97]:
exez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exez/"+str(n)+".dat")
    exez_.append(dat)
    
exez=np.array(exez_)

jk_exez = Jackknife(exez.size, binsize)
jk_exez.set(simple_mean, exez)
jk_exez.do_it()

print(format_print_w_exact_matrix(exez_corr_numeric(beta, h, kx,ky,kz), jk_exez.mean(), jk_exez.err()))

[['0.163(48) exact:0.1333 [0.61 sigma]'
  '-0.013(57) exact:0.0387 [0.90 sigma]'
  '-0.075(40) exact:0.0199 [2.36 sigma]']
 ['-0.013(36) exact:0.0226 [0.96 sigma]'
  '0.188(35) exact:0.0127 [4.94 sigma]'
  '0.050(53) exact:0.0127 [0.71 sigma]']
 ['0.175(59) exact:0.1333 [0.70 sigma]'
  '-0.013(46) exact:0.0199 [0.71 sigma]'
  '0.050(45) exact:0.0387 [0.25 sigma]']]


In [98]:
eyey_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/eyey/"+str(n)+".dat")
    eyey_.append(dat)
    
eyey=np.array(eyey_)

jk_eyey = Jackknife(eyey.size, binsize)
jk_eyey.set(simple_mean, eyey)
jk_eyey.do_it()

print(format_print_w_exact_matrix(eyey_corr_numeric(beta, h, kx,ky,kz), jk_eyey.mean(), jk_eyey.err()))

[['1.0 exact:1.0' '-0.113(69) exact:0.0281 [2.03 sigma]'
  '0.000(55) exact:0.0281 [0.51 sigma]']
 ['-0.087(56) exact:0.1333 [3.97 sigma]'
  '-0.025(51) exact:0.0216 [0.91 sigma]'
  '-0.075(45) exact:0.0226 [2.18 sigma]']
 ['0.163(56) exact:0.1333 [0.52 sigma]'
  '0.250(52) exact:0.0226 [4.38 sigma]'
  '-0.087(56) exact:0.0216 [1.96 sigma]']]


In [99]:
eyez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/eyez/"+str(n)+".dat")
    eyez_.append(dat)
    
eyez=np.array(eyez_)

jk_eyez = Jackknife(eyez.size, binsize)
jk_eyez.set(simple_mean, eyez)
jk_eyez.do_it()

print(format_print_w_exact_matrix(eyez_corr_numeric(beta, h, kx,ky,kz), jk_eyez.mean(), jk_eyez.err()))

[['0.013(31) exact:0.1089 [3.14 sigma]'
  '-0.025(70) exact:0.0181 [0.61 sigma]'
  '-0.150(26) exact:0.1089 [9.84 sigma]']
 ['0.062(56) exact:0.0387 [0.42 sigma]'
  '-0.013(31) exact:0.0127 [0.82 sigma]'
  '-0.113(63) exact:0.0209 [2.11 sigma]']
 ['0.075(56) exact:0.0209 [0.96 sigma]'
  '-0.025(58) exact:0.0127 [0.65 sigma]'
  '-0.050(53) exact:0.0387 [1.69 sigma]']]


In [100]:
ezez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/ezez/"+str(n)+".dat")
    ezez_.append(dat)
    
ezez=np.array(ezez_)

jk_ezez = Jackknife(ezez.size, binsize)
jk_ezez.set(simple_mean, ezez)
jk_ezez.do_it()

print(format_print_w_exact_matrix(ezez_corr_numeric(beta, h, kx,ky,kz), jk_ezez.mean(), jk_ezez.err()))

[['1.0 exact:1.0' '0.225(33) exact:0.0167 [6.35 sigma]'
  '-0.025(61) exact:0.0167 [0.68 sigma]']
 ['-0.025(51) exact:0.0216 [0.91 sigma]'
  '-0.050(45) exact:0.0532 [2.31 sigma]'
  '-0.113(24) exact:0.0170 [5.48 sigma]']
 ['-0.013(57) exact:0.0216 [0.60 sigma]'
  '-0.113(50) exact:0.0170 [2.60 sigma]'
  '-0.025(51) exact:0.0532 [1.53 sigma]']]
